In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
import os, sys, yaml
USER = os.getenv('USER')
from dscim.menu.simple_storage import Climate

In [ ]:
# sectors=['agriculture_clipped', 'mortality_clipped', 'energy_clipped', 'labor_clipped', 'AMEL_clipped', "CAMEL_clipped"]
sectors=['CAMEL_clipped']

In [ ]:
def scc_hist(
    sector,
    kind,
    clipped=True,
    recipe='risk_aversion',
    disc = 'euler_ramsey',
    random=False,
    gmst_drop=False
):

    output = "/mnt/CIL_integration/plots/rff_diagnostics/scc_distributions"

    if kind=='epa_mask':
        indir = f"/mnt/CIL_integration/rff_new_wp_formula/CH4_CO2_Fossil_N2O/{sector}/2020/unmasked_None/"
    elif kind=='gdppc_mask':
        indir=f'/mnt/CIL_integration/rff_new_wp_formula/CH4_CO2_Fossil_N2O/{sector}/2020/gdppc_emissions_q0.01_q0.99'
    elif 'ssp' in kind:
        indir = f'/mnt/CIL_integration/menu_results_AR6_bc39_epa_vsl_continuous/{sector}/2020/unmasked'

    near_term = {
        # 0.03: (1.567899395, 0.00770271076),
        0.025 : (1.421158116, 0.00461878399),
        0.02 : (1.244459066, 0.00197263997),
        0.015 : (1.016010255, 9.149608e-05)
    }

    sccs = []

    for rate, eta_rho in near_term.items():
        df = xr.open_dataset(
            f"{indir}/{recipe}_{disc}_eta{eta_rho[0]}_rho{eta_rho[1]}_uncollapsed_sccs.nc4"
        ).sel(weitzman_parameter='0.5', gas='CO2_Fossil')

        if kind=='epa_mask':
            rffsp_values = pd.read_csv("/shares/gcp/integration/rff/rffsp_trials.csv").rffsp_id.unique()
            df = df.sel(rff_sp=rffsp_values)

        if random==True:
            # create random variable
            df['random'] = xr.DataArray(
            data=np.random.rand(*list(df['uncollapsed_sccs'].shape)),
            dims=df['uncollapsed_sccs'].dims,
            coords=df['uncollapsed_sccs'].coords
        )
            # random selection of one simulation for each rff_sp
            df = df.uncollapsed_sccs.sel(simulation=df['random'].idxmax('simulation'), drop=True)
        else:
            df = df.uncollapsed_sccs

        if gmst_drop==True:
            if 'ssp' in kind:
                config = f'/home/{USER}/repos/integration/configs/epa_tool_config-histclim_AR6.yaml'
            else:
                config = f'/home/{USER}/repos/integration/configs/rff_config_all_gases.yaml'

            with open(config) as config_file:
                params = yaml.full_load(config_file)
                params['climate'].update({
              'gmst_fair_path' : f"/shares/gcp/integration/rff/climate/stacked/ar6_rff_iter0-19_fair162_all_gases_control_pulse_2020_temp_v5.02_newformat_Jan222022.nc"
        })

            anomalies = Climate(**params['climate']).fair_pulse.temperature.sel(year=2300, gas='CO2_Fossil', drop=True)

            df = df.where(anomalies >= 2, drop=True)

        df = df.to_dataframe()
        df['Ramsey discount rate'] = rate
        sccs.append(df.reset_index())

    data = pd.concat(sccs).reset_index()

    subsets = {}
    if kind=='ssp_unique':
        for s,m in product(data.ssp.unique(), data.model.unique()):
            subsets[f"{m}_{s}"] = data.loc[(data.ssp == s) & (data.model == m)]
    else:
        subsets[''] = data

    fig, ax = plt.subplots(2,len(subsets),figsize=(10*len(subsets),8), sharex=True, gridspec_kw={'height_ratios' : (3,1)}, squeeze=False)

    for i, s in enumerate(subsets.items()):

        means = s[1].groupby('Ramsey discount rate').mean().uncollapsed_sccs
        table=s[1].groupby('Ramsey discount rate').uncollapsed_sccs.describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).T
    
        pal = sns.color_palette('husl', 3)

        sns.kdeplot(data=s[1],
                    x='uncollapsed_sccs',
                    hue='Ramsey discount rate',
                    ax=ax[0][i],
                    palette=pal,
                   )

        sns.boxplot(data=s[1],
                    x='uncollapsed_sccs',
                    y='Ramsey discount rate',
                    hue='Ramsey discount rate',
                    orient='h',
                    showfliers=False,
                    whis=[5, 95],
                    ax=ax[1][i],
                    palette=pal,
                   )

        for d, discrate in enumerate(means.index.values):
            ax[0][i].axvline(
                means[discrate],
                0, 0.75,
                color=pal[d],
                linestyle='--'
                       )

            ax[0][i].annotate(round(means[discrate],2),
                       (means[discrate], ax[0][i].get_ylim()[1]*(0.75+d*3/100)),
                       xycoords='data',
                       color=pal[d],
                      )

        for a in ax.flatten():
            if clipped==True:
                a.set_xlim(
                    table.loc['1%'].min(), 
                    table.loc['99%'].max()
                )
            a.spines['top'].set_visible(False)
            a.spines['right'].set_visible(False)
            a.spines['bottom'].set_visible(False)
            a.spines['left'].set_visible(False)
            a.axes.get_yaxis().set_visible(False)

        ax[1][i].get_legend().remove()
        ax[1][i].set_xlabel("")
        start, end = ax[1][i].get_xlim()
        ax[1][i].xaxis.set_ticks((np.linspace(start, end, 12)))
        ax[0][i].set_title(s[0])

        plt.subplots_adjust(wspace=0.05, hspace=0)

    fig.suptitle(f"The Social Cost of Carbon Dioxide in 2020 by Discount Rate \n {sector}, {kind}, {recipe}, {disc}, random-{random}, clip-1-99-{clipped}, gmst_drop-{gmst_drop}")
    os.makedirs(output, exist_ok=True)
    plt.savefig(f'{output}/{sector}_{kind}_{recipe}_{disc}_uncollapsed_scc_histograms_epa_version_random-{random}-clip-{clipped}_gmst_drop-{gmst_drop}.png', dpi=300, bbox_inches='tight')
    table.to_csv(f'{output}/{sector}_{kind}_{recipe}_{disc}_uncollapsed_scc_table_epa_version_random-{random}-clip-{clipped}_gmst_drop-{gmst_drop}.csv')

In [ ]:
for sector in sectors:
    for kind in ['epa_mask', 'gdppc_mask']: # ['ssp_unique', 'ssp']:
        for recipe in ['risk_aversion', 'adding_up']:
            scc_hist(sector,
                     kind=kind,
                     recipe=recipe,
                     clipped=False,
                     # gmst_drop=True
                    )